In [ ]:
# Import required libraries
import pandas as pd
import dash
#import dash_html_components as html
from dash import html
#import dash_core_components as dcc
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown', options=[{'label':'All Sites','value':'ALL'},{'label':'CCAFS LC-40','value':'CCAFS LC-40'},{'label':'CCAFS SLC-40','value':'CCAFS SLC-40'},{'label':'KSC LC-39A','value':'KSC LC-39A'},{'label':'VAFB SLC-4E','value':'VAFB SLC-4E'}],value='ALL', placeholder='Select a Launch Site here', searchable=True),
                                html.Br(),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider', 
                                                min=0, max=10000, 
                                                step=1000, 
                                                marks={0: '0kg', 
                                                       2000: '2000kg',
                                                       4000: '4000kg',
                                                       6000: '6000kg',
                                                       8000: '8000kg',
                                                       10000: '10000kg'
                                                      }, 
                                                value=[0, 10000]
                                               ),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
            Input(component_id='site-dropdown', component_property='value'))       

def get_pie_chart(entered_site):
    filtered_df = spacex_df.groupby('Launch Site')['class'].mean().reset_index()
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Success Launches by Site')
        return fig
    else:      
        new_data = spacex_df[spacex_df['Launch Site']==str(entered_site)]['class'].value_counts().reset_index().rename(columns={'index':'class','class':'count'})
        new_data['class']=new_data['class'].astype(str)
        fig = px.pie(new_data, values='count', names='class', title='Total Success Launches for ' + str(entered_site),
                    color="class", color_discrete_map={"0":"#ff4500", "1":"#1f75fe"})
        return fig
    
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'), 
              Input(component_id='site-dropdown', component_property='value'), Input(component_id='payload-slider', component_property='value'))

def get_scatter_plot(entered_site,value):
    if entered_site == 'ALL':
        filtered_df2 = spacex_df
        filtered_df3 = filtered_df2[filtered_df2['Payload Mass (kg)']>value[0]]
        filtered_df4 = filtered_df3[filtered_df3['Payload Mass (kg)']<value[1]]
        fig = px.scatter(filtered_df4, x='Payload Mass (kg)', y='class', 
                         color='Booster Version Category',title="Correlation between payload and success for all sites")
        return fig
    else:
        #filtered_df2 = spacex_df[spacex_df['Launch Site']==str(entered_site)]
        filtered_df2 = spacex_df[spacex_df['Launch Site']==str(entered_site)]
        filtered_df3 = filtered_df2[filtered_df2['Payload Mass (kg)']>value[0]]
        filtered_df4 = filtered_df3[filtered_df3['Payload Mass (kg)']<value[1]]
        fig = px.scatter(filtered_df4, x='Payload Mass (kg)', y='class', 
                         color='Booster Version Category',title="Correlation between payload and success for "+entered_site)
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()



Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Nov/2021 15:58:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 15:58:46] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 15:58:46] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 15:58:47] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 15:58:47] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 15:58:47] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 15:58:47] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 15:59:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 15:59:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 15:59:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 15:59:44] "POST /_dash-